In [1]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image

/home/leloykun/anaconda3/envs/MMFM-Challenge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "khhuang/chart-to-table"
model = VisionEncoderDecoderModel.from_pretrained(model_name).cuda()
processor = DonutProcessor.from_pretrained(model_name)

/home/leloykun/anaconda3/envs/MMFM-Challenge/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
image_path = "data/raw_datasets/mychart/images/328_hbar_0635a64e806fee03461eddecf061fd851a801f3b482d208e5c651e7c68c69585_66.png"
input_prompt = "<data_table_generation> <s_answer>"

In [4]:
img = Image.open(image_path)
pixel_values = processor(img.convert("RGB"), random_padding=False, return_tensors="pt").pixel_values
pixel_values = pixel_values.cuda()
decoder_input_ids = processor.tokenizer(input_prompt, add_special_tokens=False, return_tensors="pt", max_length=510).input_ids.cuda()#.squeeze(0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [5]:
outputs = model.generate(
    pixel_values.cuda(),
    decoder_input_ids=decoder_input_ids.cuda(),
    max_length=model.decoder.config.max_position_embeddings,
    early_stopping=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    num_beams=4,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

In [6]:
sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
extracted_table = sequence.split("<s_answer>")[1].strip()

In [7]:
extracted_table

'TITLE | Annual net salary of an apartment in the United States in 2019, by sector &&& Group | Total | Acquired license rights &&& 30061 | 1738 | Travelmarks and trade names &&& 27438 | 5868 | Completed technology &&& 10052 | 3118 | 6286 | Completed technology &&& New York Times Company Agency && Average 1,000 B2R&A Group && Amount && Estimated && Estimated Estimated && Estimated Estimated Estimated Estimated Estimated Estimated Estimated Estimated Estimated Estimated bankruption && Estimated bankruption && Estimated bankrupt agencies and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expenditure and expendi